# SQL Assignment

In [1]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect("Db-IMDB-Assignment.db")

## Sample Code

In [3]:
%%time
# Write your sql query below

query = """
        SELECT TRIM(Movie.title) AS 'Movie_Name'
        FROM Movie
        WHERE Movie.rating < 3
        
        """

q = pd.read_sql_query(query, conn)    
print(q.shape)
q.head()

(85, 1)
Wall time: 154 ms


## Q1 --- List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year.

In [20]:
# %%time
# Write your sql query below

query = """
        
        SELECT p.Name, m.title, m.year
        FROM Person p JOIN M_Director md ON p.PID=md.PID JOIN Movie m ON md.MID=m.MID
        WHERE m.MID IN(
            SELECT MID FROM M_Genre WHERE GID IN(
                SELECT GID FROM Genre WHERE Name like '%Comedy%'))
        AND m.year%4==0 AND m.year%100!=0
        
        """

# Samreen----------------------------------------
#         SELECT p.Name, m.title, m.year
#         FROM Person p JOIN M_Director md ON p.PID=md.PID JOIN Movie m ON md.MID=m.MID
#         WHERE m.MID IN(
#             SELECT MID FROM M_Genre mg Join Genre g On mg.GID=g.GID Join Movie mv On mg.MID=mv.MID
#              WHERE g.Name like '%Comedy%'AND mv.year%4==0 AND m.year%100!=0)
# """
q1 = pd.read_sql_query(query, conn)    
print(q1.shape)
q1.head()

(212, 3)


,Name,title,year
0,Milap Zaveri,Mastizaade,2016
1,Danny Leiner,Harold & Kumar Go to White Castle,2004
2,Anurag Kashyap,Gangs of Wasseypur,2012
3,Frank Coraci,Around the World in 80 Days,2004
4,Griffin Dunne,The Accidental Husband,2008


## Q2 --- List the names of all the actors who played in the movie 'Anand' (1971)

In [5]:
%%time
# Write your sql query below

query = """
        SELECT p.Name
        FROM Movie m JOIN M_Cast mc ON m.MID=TRIM(mc.MID) JOIN Person p ON p.PID=TRIM(mc.PID)
        WHERE m.title='Anand' AND year=1971
        
        """

q2 = pd.read_sql_query(query, conn)    
print(q2.shape)
print(q2.head())

(17, 1)
                Name
0      Rajesh Khanna
1   Amitabh Bachchan
2      Sumita Sanyal
3         Ramesh Deo
4          Seema Deo
Wall time: 257 ms


In [6]:
%%time
# Write your sql query below

query = """
        SELECT Name FROM Person WHERE PID IN(
            SELECT TRIM(PID) FROM M_Cast WHERE MID IN(
                (SELECT MID FROM Movie WHERE title='Anand' and year=1971)))
        
        """

q2 = pd.read_sql_query(query, conn)    
print(q2.shape)
print(q2.head())

(17, 1)
                Name
0   Amitabh Bachchan
1      Rajesh Khanna
2      Sumita Sanyal
3         Ramesh Deo
4          Seema Deo
Wall time: 56 ms


## Q3 --- List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)

In [7]:
# %%time
# Write your sql query below

query = """
       SELECT Name
       FROM(
           SELECT Name, PID FROM Person WHERE PID IN(
                SELECT TRIM(PID) FROM M_Cast
                WHERE MID IN (SELECT MID FROM Movie WHERE year<1970)
           )
           
           INTERSECT
           
           SELECT Name, PID FROM Person WHERE PID IN(
                SELECT TRIM(PID) FROM M_Cast
                WHERE MID IN (SELECT MID FROM Movie WHERE year>1990)
           )
        )
        """
"""---samreen--Please check the syntax of not between and its inclusive propertt of it either it include year 1970 an 1990 or not--

               SELECT Name FROM Person WHERE PID IN(SELECT TRIM(PID) FROM M_Cast
                                                 WHERE MID IN (SELECT MID FROM Movie WHERE year NOT BETWEEN 1970 to 1990)
           
           
        """
q3 = pd.read_sql_query(query, conn)    
print(q3.shape)
q3.head()

(333, 1)


,Name
0,A.K. Hangal
1,Aachi Manorama
2,Abbas
3,Abdul
4,Abhi Bhattacharya


## Q4 --- List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.

In [8]:
# %%time

query = """
        SELECT p.Name, COUNT(title) AS Number_of_Movies
        FROM Person p JOIN M_Director md ON p.PID=md.PID JOIN Movie m ON m.MID=md.MID 
        GROUP BY p.Name
        HAVING Number_of_Movies >= 10
        ORDER BY Number_of_Movies DESC
        
        """

q4 = pd.read_sql_query(query, conn)    
print(q4.shape)
q4.head()

(58, 2)


,Name,Number_of_Movies
0,David Dhawan,39
1,Mahesh Bhatt,36
2,Ram Gopal Varma,30
3,Priyadarshan,30
4,Vikram Bhatt,29


## Q5.a --- For each year, count the number of movies in that year that had only female actors.

In [9]:
# %%time
# Write your sql query below

query = """
        SELECT year, COUNT(title) AS Number_of_Movies
        FROM Person p JOIN M_Cast mc ON p.PID=TRIM(mc.PID) JOIN Movie m ON m.MID=mc.MID
        WHERE p.Gender='Female'
        GROUP BY year
        
        """

q5a = pd.read_sql_query(query, conn)    
print(q5a.shape)
q5a.head()

(125, 2)


,year,Number_of_Movies
0,1931,3
1,1936,19
2,1939,18
3,1941,7
4,1943,3


## Q5.b --- Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer.

In [10]:
# %%time
# Write your sql query below

query = """
        SELECT t1.year, (t1.Number_of_Movies*100.0/t2.Number_of_Movies) AS [% of Female Actors Movies], t2.Number_of_Movies
        FROM
        (SELECT year, COUNT(title) AS Number_of_Movies
        FROM Person p JOIN M_Cast mc ON p.PID=TRIM(mc.PID) JOIN Movie m ON m.MID=mc.MID
        WHERE p.Gender='Female'
        GROUP BY year) AS t1
        JOIN
        (SELECT year, COUNT(title) AS Number_of_Movies
        FROM Person p JOIN M_Cast mc ON p.PID=TRIM(mc.PID) JOIN Movie m ON m.MID=mc.MID
        GROUP BY year) AS t2
        ON t1.year=t2.year
        """

q5b = pd.read_sql_query(query, conn)    
print(q5b.shape)
q5b.head()

(125, 3)


,year,% of Female Actors Movies,Number_of_Movies
0,1931,33.333333,9
1,1936,40.425532,47
2,1939,40.000000,45
3,1941,12.962963,54
4,1943,21.428571,14


## Q6 --- Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once.

In [11]:
# %%time

query = """
        SELECT m.title, COUNT(DISTINCT p.name) 'cast_size'
        FROM Person p JOIN M_Cast mc ON p.PID=TRIM(mc.PID) JOIN Movie m ON m.MID=mc.MID
        GROUP BY m.title
        ORDER BY cast_size DESC
        """

q6 = pd.read_sql_query(query, conn)    
print(q6.shape)
q6.head()

(3342, 2)


,title,cast_size
0,Ocean's Eight,238
1,Apaharan,233
2,Gold,214
3,My Name Is Khan,213
4,Captain America: Civil War,191


## Q7 --- A decade is a sequence of 10 consecutive years. For example, say in your database you have movie information starting from 1965. Then the first decade is 1965, 1966, ..., 1974; the second one is 1967, 1968, ..., 1976 and so on. Find the decade D with the largest number of films and the total number of films in D.

In [12]:
# %%time

query = """
         
select Max(total_no_of_movies_in_a_decade)
from  (select count(title) as total_no_of_movies_in_a_decade,decade, decade+9
       from (select ((year/10) * 10) as decade, title from Movie)  
       group by decade
       order by total_no_of_movies_in_a_decade DESC)
    
"""    
      

q7 = pd.read_sql_query(query, conn)    
print(q7.shape)
# q7.tail(130)
q7.head()

(1, 1)


,Max(total_no_of_movies_in_a_decade)
0,1018


In [13]:
# PRAGMA table_info([Movie]) to see information about table
# CAST(version as integer)

## Q8 --- Find all the actors that made more movies with Yash Chopra than any other director.

In [18]:
# %%time
# Write your sql query below

query = """
SELECT t1.pname
FROM(
    SELECT pname,dname, count(dname) count
    FROM(
        SELECT p.PID pid, p.Name pname, p1.Name dname
        FROM Person p JOIN M_Cast mc ON p.PID=TRIM(mc.PID)
            JOIN Movie m ON mc.MID=m.MID
            JOIN M_Director md ON m.MID=md.MID
            JOIN Person p1 ON md.PID=p1.PID
        WHERE p.Gender = 'Male'
        )
    GROUP BY pname, dname 
    ) t1
WHERE TRIM(t1.dname) == 'Yash Chopra'
    AND t1.count >= (
        SELECT t2.count
        FROM(
            SELECT pname, dname, count(dname) count
            FROM(
                SELECT p.PID pid, p.Name pname, p1.Name dname
                FROM Person p JOIN M_Cast mc ON p.PID=TRIM(mc.PID)
                    JOIN Movie m ON mc.MID=m.MID
                    JOIN M_Director md ON m.MID=md.MID
                    JOIN Person p1 ON md.PID=p1.PID
                WHERE p.Gender = 'Male'
            )
        GROUP BY  pname, dname
        )
    t2)
 """


q8 = pd.read_sql_query(query, conn)    
print(q8.shape)
q8.head()

(271, 1)


,pname
0,A.K. Hangal
1,Aamir Khan
2,Ajit
3,Akhilendra Mishra
4,Akshay Kumar


## Q9 --- The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2.

In [17]:
# %%time
# Write your sql query below

query = """

SELECT DISTINCT(p4.Name)
FROM Person p4 JOIN M_Cast mc4 ON p4.PID=TRIM(mc4.PID) JOIN Movie m4 ON mc4.MID=m4.MID
WHERE m4.title IN (
    SELECT DISTINCT(m3.title)
    FROM Person p3 JOIN M_Cast mc3 ON p3.PID=TRIM(mc3.PID) JOIN Movie m3 ON mc3.MID=m3.MID
    WHERE p3.Name IN(
        SELECT DISTINCT(p2.Name)
        FROM Person p2 JOIN M_Cast mc2 ON p2.PID=TRIM(mc2.PID) JOIN Movie m2 ON mc2.MID=m2.MID
        WHERE m2.title in(
            SELECT m1.title
            FROM Person p1 JOIN M_Cast mc1 ON p1.PID=TRIM(mc1.PID) JOIN Movie m1 ON mc1.MID=m1.MID
            WHERE p1.name like '%Rukh khan%'
        )
        AND p2.Name not like '%Rukh khan%'
    )
)               
  
"""    

q9 = pd.read_sql_query(query, conn)    
print(q9.shape)
q9.head()

(27028, 1)


,Name
0,Kristin Scott Thomas
1,Derek Jacobi
2,Nick Frost
3,Walton Goggins
4,Michael Obiora
